In [ ]:
from pyspark.sql import SparkSession
import mlflow.spark
from mlflow.tracking import MlflowClient

In [ ]:
Gateway = "[여기에 게이트웨이 주소를 입력해주세요]"

In [ ]:
import os
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://{Gateway}:9000"
os.environ["MLFLOW_TRACKING_URI"] = f"http://{Gateway}:5001"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "miniostorage"

In [ ]:
# SparkSession 생성
spark = (SparkSession
         .builder
         .appName("SparkMlflowExampleApp")
         .getOrCreate())

In [ ]:
client = MlflowClient()

In [ ]:
current_experiment=dict(mlflow.get_experiment_by_name('BOAZ-Test'))
experiment_id=current_experiment['experiment_id']

In [ ]:
# 실험 환경 가져오기
experiment = client.get_experiment(experiment_id)
experiment

In [ ]:
# 실행 검색
client = MlflowClient()
runs = client.search_runs(experiment_ids = experiment.experiment_id,
                         order_by= ["metrics.rmse"], # rmse가 작은 모델부터 검색
                          max_results = 3)
run_id = runs[0].info.run_id
run_name = runs[0].info.run_name
print(run_id)
print(run_name)
print(runs[0].data.metrics)
print(runs[0].data.params)

In [ ]:
# 모델 로드
pipelineModel = mlflow.spark.load_model(f"runs:/{run_id}/rf")

In [ ]:
# 예측
inputDF = spark.read.parquet("./data/sf-airbnb-clean.parquet")
predDF = pipelineModel.transform(inputDF)

In [ ]:
predDF.toPandas()[['prediction']].head(3)

In [ ]:
spark.stop()